# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Workspace and Experiment
ws = Workspace.from_config()

experiment_name = 'hyperdrive_experiment'
experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-225458
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-225458


In [3]:
# Creating a Compute Cluster specific for HyperDrive Run
cluster_name = "hyperdrive-cluster"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_V2', 
        max_nodes=4
    )

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
from azureml.core import Environment, ScriptRunConfig

# Specification of an early termination policy
early_termination_policy = BanditPolicy(
    evaluation_interval=1,
    slack_factor = 0.1,  
    delay_evaluation=4
)

# Specify parameter sampler
param_sampling = RandomParameterSampling( 
    {
    "--C" : uniform(0.1,1),
    "--max_iter" : choice(50, 100, 150, 200)
    }
)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(
    name='sklearn-env', 
    file_path='conda_dependencies.yml'
)

# Create your estimator and hyperdrive config
if "training" not in os.listdir():
        os.mkdir("./training")

estimator = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_cluster,
    environment=sklearn_env
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(
    config=hyperdrive_run_config,
    show_output=True
)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa
Web View: https://ml.azure.com/runs/HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-225458/workspaces/quick-starts-ws-225458&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-11T13:47:46.477904][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-11T13:47:48.9978089Z][SCHEDULER][INFO]Scheduling job, id='HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa_0' 
[2023-02-11T13:47:49.0867268Z][SCHEDULER][INFO]Scheduling job, id='HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa_1' 
[2023-02-11T13:47:49.2243094Z][SCHEDULER][INFO]Scheduling job, id='HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa_2' 
[2023-02-11T13:47:49.267181][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-02-11T13:47:49.3391981Z][SCHEDULER][INFO]Scheduling job, id='HD_d57dd897-33ac-4a25-8eca-4b89fb26f6

{'runId': 'HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa',
 'target': 'hyperdrive-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-11T13:47:44.554501Z',
 'endTimeUtc': '2023-02-11T13:53:04.709079Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '20fe45e5-c173-46af-9ad0-dd551eee9632',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.7676767676767676',
  'best_child_run_id': 'HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telem

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
# Get the best model
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()
hyperdrive_parameter_values = best_hyperdrive_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Accuracy: ', best_hyperdrive_run_metrics['Accuracy'])
print('\n Hyperparameters: ', hyperdrive_parameter_values)

Best Run Id:  HD_d57dd897-33ac-4a25-8eca-4b89fb26f6fa_0

 Accuracy:  0.7676767676767676

 Hyperparameters:  ['--C', '0.8842527732891214', '--max_iter', '150']


In [58]:
# Save the best model as joblib and pkl file
joblib.dump(hyperdrive_parameter_values, 'best-hyperdrive-model.pkl')

['best-hyperdrive-model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [59]:
## See for more details: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb

model = Model.register(
    workspace=ws,
    model_name = "best-hyperdrive-model", 
    model_path = './best-hyperdrive-model.pkl')

#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'conda_dependencies.yml')
print(model.name, model.id, model.version, sep='\t')

Registering model best-hyperdrive-model
best-hyperdrive-model	best-hyperdrive-model:2	2


In [48]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
# Deploy the model
inference_config = InferenceConfig(
    entry_script='score_hyperdrive.py', 
    environment=sklearn_env
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    auth_enabled=True,
    tags = {'name':'failure Prediction'},
    description='Heart failure prediction model'
)

service = model.deploy(
    workspace=ws,
    name="hyperdrive-deploy4",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-11 15:32:24+00:00 Creating Container Registry if not exists.
2023-02-11 15:32:26+00:00 Use the existing image.
2023-02-11 15:32:26+00:00 Generating deployment configuration.
2023-02-11 15:32:27+00:00 Submitting deployment to compute.
2023-02-11 15:32:31+00:00 Checking the status of deployment hyperdrive-deploy4..
2023-02-11 15:34:35+00:00 Checking the status of inference endpoint hyperdrive-deploy4.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [49]:
# Enable Logging and check state
service.update(enable_app_insights=True)
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [60]:
import json
import requests

# URL for the web service
scoring_uri = 'http://c24f0b51-99d8-4fb2-a152-0f9712a0b555.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'YBCGm9XgJDsd1x4ydcslqrkEjQwr2Ypd'

data = {
        "data": [
            {
                'age': 75.0, 
                'anaemia': 0.0, 
                'creatinine_phosphokinase': 582.0,
                'diabetes': 0.0,
                'ejection_fraction': 20.0,
                'high_blood_pressure': 1.0,
                'platelets': 265000.0,
                'serum_creatinine': 1.9,
                'serum_sodium': 130.0,
                'sex': 1.0,
                'smoking': 0.0,
                'time': 4.0          
            },
        ]   
}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(
    scoring_uri, 
    input_data, 
    headers=headers
)
    
print("prediction ", resp.json())
print(output)

prediction  'list' object has no attribute 'predict'
'list' object has no attribute 'predict'


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
compute_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

